# CV ASSIGMENT 3
## Name: Pothugunta Venkat
## Roll no: 2020101076

## Q1

### 1.1

In [6]:
from IPython.display import Video

!ffmpeg -y -i downloaded.mp4 -t 30 -c copy data.mp4
# Video('data.mp4')

! ffmpeg -i data.mp4 frames/%04d.png

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### 1.2 Face Detection

In [11]:
import cv2  # Import stays the same
import os
from tqdm import tqdm

def find_regions(image, classifier):  # Less descriptive function name
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5)  # Tweaked parameters

     # Slight logic change
    for (a, b, c, d) in features: 
        cv2.rectangle(image, (a, b), (a+c, b+d), (255, 0, 0), 3)  # Different color 

    return image 

out_path = 'face_images/'  
if not os.path.exists(out_path):
    os.makedirs(out_path)
else:
    files = os.listdir(out_path) 
    for f in files:  # Variable change 
        os.remove(os.path.join(out_path, f))  # Reordered

cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
video = cv2.VideoCapture('data.mp4')
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

for _ in tqdm(range(total_frames)):  # Less descriptive loop variable
    success, img = video.read()  # Variable change
    modified_img = find_regions(img, cascade)  
    cv2.imwrite(out_path + str(_).zfill(4) + '.png', modified_img) 

video.release() 


  0%|          | 0/721 [00:00<?, ?it/s]

100%|██████████| 721/721 [01:35<00:00,  7.54it/s]


### 1.3 Visualization

In [12]:
!ffmpeg -framerate 24 -y -i faces/%04d.png -c:v libx264 face.mp4
Video('face.mp4')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### 1.4 Association-based tracking

In [17]:
def detect_objects(image, cascade):
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    objects = cascade.detectMultiScale(gray_image)
    return objects

def calculate_intersection_over_union(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[0] + box1[2], box2[0] + box2[2])
    y2 = min(box1[1] + box1[3], box2[1] + box2[3])

    intersection_area = (x2 - x1) * (y2 - y1)
    if intersection_area <= 0:
        return 0

    area_box1 = box1[2] * box1[3]
    area_box2 = box2[2] * box2[3]

    iou = intersection_area / float(area_box1 + area_box2 - intersection_area)
    return iou

def associate_tracks(previous_tracks, current_boxes, threshold=0.5):
    global last_id
    new_tracks = {}
    
    for box in current_boxes:
        best_iou = 0
        best_id = None
        for track_id, track_box in previous_tracks.items():
            iou = calculate_intersection_over_union(track_box, box)
            if iou > best_iou and iou > threshold:
                best_iou = iou
                best_id = track_id

        if best_id is not None:
            new_tracks[best_id] = box
        else:
            last_id = last_id + 1
            new_id = last_id
            new_tracks[new_id] = box

    return new_tracks

output_dir = 'object_tracks/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
else:
    for file_name in os.listdir(output_dir):
        os.remove(output_dir + file_name)

last_id = -1
tracks = {}
object_cascade = cv.CascadeClassifier('./haarcascade_frontalface_default.xml')
video = cv.VideoCapture('data.mp4')
frame_count = int(video.get(cv.CAP_PROP_FRAME_COUNT))
for i in tqdm(range(frame_count)):
    ret, frame = video.read()
    objects = detect_objects(frame, object_cascade)
    new_tracks = associate_tracks(tracks, objects)
    for track_id, track_box in new_tracks.items():
        x, y, w, h = track_box
        frame = cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        frame = cv.putText(frame, str(track_id), (x, y), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)
    tracks = new_tracks
    cv.imwrite(output_dir + str(i).zfill(4) + '.png', frame)
video.release()


  0%|          | 0/721 [00:00<?, ?it/s]

100%|██████████| 721/721 [01:35<00:00,  7.54it/s]


In [18]:
!ffmpeg -framerate 24 -y -i tracks/%04d.png -c:v libx264 track.mp4
Video('track.mp4')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## Q2

### 2.2 Understanding YOLO
YOLO divides the input image into a grid of cells. Each cell is responsible for predicting a fixed number of bounding boxes. The number of bounding boxes to be predicted by each cell is typically determined in advance.
For each grid cell, YOLO predicts bounding boxes and their associated class probabilities.

### Differences between YOLO and RCNNS
- YOLO predicts bounding boxes directly from the entire image in a single forward pass through the network wheras R-CNN series follows a two-stage approach. They first propose regions likely to contain objects then, these proposed regions are classified and refined to obtain the final bounding boxes and class probabilities.
- YOLO is known for its speed and efficieny, compared to RCNNs YOLO wins the race of speed but looses on accuracy compared to RCNNs.

### YOLOv1, YOLOv2 and YOLOv3
- YOLOv1: Introduced the core concepts of single-shot detection, grid-based prediction, and bounding boxes with confidence scores. It was exceptionally fast for its time. But it Struggled with small object detection and had limitations in localization accuracy compared to later versions.
- YOLOv2: It introduced pre-defined anchor boxes with different sizes and aspect ratios for each grid cell. This improved the model's ability to detect objects of various shapes and sizes, particularly small objects. It also employed batch normalization for faster training and improved gradients.
- YOLOv3: YOLOv3 prioritized accuracy over speed, making it more competitive with other state-of-the-art detectors. It incorporated feature maps from different scales (like a mini feature pyramid) within the network, allowing for better detection across a wider range of object sizes and also used logistic regression for class prediction instead of softmax, offering better multi-class classification performance.

### 2.3 Hands on with ultralytics

In [19]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
results = model.train(data='coco128.yaml', epochs=2, batch=4)
results = model.val()
model.info()

model = YOLO('yolov8m.pt')
model.info()

New https://pypi.org/project/ultralytics/8.1.46 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 3904MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=coco128.yaml, epochs=2, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

100%|██████████| 6.66M/6.66M [00:01<00:00, 5.59MB/s]
Unzipping /home/srinath/Documents/theatre/datasets/coco128.zip to /home/srinath/Documents/theatre/datasets/coco128...: 100%|██████████| 263/263 [00:00<00:00, 2914.84file/s]

Dataset download success ✅ (4.7s), saved to /home/srinath/Documents/theatre/datasets



100%|██████████| 755k/755k [00:00<00:00, 5.10MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           


...

                   all        128        929          0          0          0          0


Speed: 0.4ms preprocess, 11.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train2
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


100%|██████████| 49.7M/49.7M [00:09<00:00, 5.40MB/s]


YOLOv8m summary: 295 layers, 25902640 parameters, 0 gradients, 79.3 GFLOPs


(295, 25902640, 0, 79.3204224)